## World Wide Products!

This week, we will be looking at forecasting models using a sample dataset that deals with demand forecasting. 

In [1]:
# import
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("../data/external/Historical Product Demand.csv")
df.head(5)

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012/7/27,100
1,Product_0979,Whse_J,Category_028,2012/1/19,500
2,Product_0979,Whse_J,Category_028,2012/2/3,500
3,Product_0979,Whse_J,Category_028,2012/2/9,500
4,Product_0979,Whse_J,Category_028,2012/3/2,500


In [7]:
# convert date to datetime
df['Date'] = pd.to_datetime(df['Date'])
df.head(5)

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012-07-27,100
1,Product_0979,Whse_J,Category_028,2012-01-19,500
2,Product_0979,Whse_J,Category_028,2012-02-03,500
3,Product_0979,Whse_J,Category_028,2012-02-09,500
4,Product_0979,Whse_J,Category_028,2012-03-02,500


In [8]:
# create new year and month features 
df['Year'], df['Month'] = df['Date'].dt.year, df['Date'].dt.month
df.head(5)

,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Year,Month
0,Product_0993,Whse_J,Category_028,2012-07-27,100,2012.0,7.0
1,Product_0979,Whse_J,Category_028,2012-01-19,500,2012.0,1.0
2,Product_0979,Whse_J,Category_028,2012-02-03,500,2012.0,2.0
3,Product_0979,Whse_J,Category_028,2012-02-09,500,2012.0,2.0
4,Product_0979,Whse_J,Category_028,2012-03-02,500,2012.0,3.0
